### Auth

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-shrgnn')

### ROI

In [ ]:
from pyproj import Proj, transform

# Define the EPSG code for EPSG:32637 (UTM Zone 37N) and WGS84
epsg_32637 = Proj(init='epsg:32637')
wgs84 = Proj(init='epsg:4326')

# Coordinates in EPSG:32637
x1, y1 = 647500, 5766000
x2, y2 = 651000, 5770000

# Convert EPSG:32637 coordinates to WGS84
lon1, lat1 = transform(epsg_32637, wgs84, x1, y1)
lon2, lat2 = transform(epsg_32637, wgs84, x2, y2)

region = ee.Geometry.Polygon(
  [[[lon1, lat2],
    [lon1, lat1],
    [lon2, lat1],
    [lon2, lat2]]], None, False)


/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
<ipython-input-3-1eea8e714a21>:12: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon1, lat1 = transform(epsg_32637, wgs84, x1, y1)
<

NDWI

In [ ]:
import ee

# Initialize the Earth Engine library
ee.Initialize()

# Define the region of interest (ROI)
roi = region
# Function to compute NDWI
def computeNDWI(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return image.addBands(ndwi)

# Sentinel-2 collection
s2_sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(roi) \
    .filterDate('2019-04-01', '2023-05-05') \
    .map(computeNDWI)

# Filter for April images
april_images = s2_sr.filter(ee.Filter.calendarRange(4, 4, 'month'))

# Compute median NDWI for each April
median_ndwi_april = april_images.median()

# Export the median NDWI image to Google Drive
task = ee.batch.Export.image.toDrive(
    image=median_ndwi_april.toFloat(),
    description='median_ndwi_april',
    folder='GEE_exports',
    region=roi,
    scale=10  # Adjust the scale as needed
)

# Start the export task
task.start()

# Print task status
print('Exporting median NDWI image to Google Drive...')


Exporting median NDWI image to Google Drive...
